In [ ]:
# required libraries & dataset

#!curl -L "https://www.dropbox.com/s/ypvbhkgt4sb40sy/dataset_b.zip?dl=0" > dataset_b.zip; unzip dataset_b.zip; rm dataset_b.zip
#!pip install livelossplot

In [ ]:
# Importing dependencies

import os
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import keras
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization
from keras.layers import concatenate, GlobalAvgPool2D, Dropout, DepthwiseConv2D, ReLU, Activation
from keras.models import Model
import keras.backend as K

In [ ]:
#Setting TPU or CPU/GPU strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
# Image size and Input shape 
IMG_SIZE = 224
width = 224
height = 224
input_shape = (height, width, 3)
seed = 1432
class_mode = 'binary'
n_classes = 1

In [ ]:
# Training/testing directory. Change as needed
TRAIN_DIR = "/content/train"
TEST_DIR = "/content/test"
VALID_DIR = "/content/valid"
batch_size = 32

In [ ]:
train_imgs = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]
test_imgs = [TEST_DIR+i for i in os.listdir(TEST_DIR)]
valid_imgs = [VALID_DIR+i for i in os.listdir(VALID_DIR)]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        seed=seed,
        class_mode=class_mode)

validation_generator = test_datagen.flow_from_directory(
        VALID_DIR,
        target_size=(height, width),
        batch_size=batch_size,
        seed=seed,
        class_mode=class_mode)

classnames = train_generator.class_indices

print(classnames)

In [ ]:
#Image augmentation
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
#SqueezeNet Model
def SqueezeNet(input_shape, num_classes):
  def fire(x, fs, fe):
    s = Conv2D(fs, 1, activation='relu')(x)
    e1 = Conv2D(fe, 1, activation='relu')(s)
    e3 = Conv2D(fe, 3, padding='same', activation='relu')(s)
    output = concatenate([e1,e3])
    return output

  input = Input(input_shape)
  x = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = fire(x, 16, 64)
  x = fire(x, 16, 64)
  x = fire(x, 16, 128)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = fire(x, 32, 128)
  x = fire(x, 48, 192)
  x = fire(x, 48, 192)
  x = fire(x, 64, 256)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = fire(x, 64, 256)
  x = Conv2D(n_classes, 1)(x)

  x = GlobalAvgPool2D()(x)
  output = Activation('sigmoid')(x)

  model = Model(input, output)
  return model

In [ ]:
K.clear_session()
model = SqueezeNet(input_shape, n_classes)

In [ ]:
#Freezing pre-trained layers
for layers in (model.layers)[:16]:
  layers.trainable= False

In [ ]:
print('Number of trainable layers:', len(model.trainable_weights))

In [ ]:
#Modifying last layer, needs rework
X=Flatten()(model.output)
predictions=Dense(1,activation='sigmoid')(X)
squeezenetmodel_final= Model(model.input, predictions)

In [ ]:
#Model Summary
squeezenetmodel_final.summary()

In [ ]:
#Compilation

learning_rate = 0.0001
epochs= 100
optimizer = tf.keras.optimizers.Adam(lr=learning_rate, decay=learning_rate/epochs)

squeezenetmodel_final.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()])

squeezenetmodel_final.summary()

In [ ]:
#Training

from livelossplot import PlotLossesKeras
from livelossplot.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("squeezenet_rev230131.h5",
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=False,
                             save_weights_only=False,
                             mode='auto',
                             save_freq=1)

early = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=10,
                      verbose=1, 
                      restore_best_weights=True,
                      mode='auto')

In [ ]:
%%time

squeezenet_hist=squeezenetmodel_final.fit(train_generator,
                        epochs=epochs,
                        validation_data=validation_generator, 
                        callbacks= [checkpoint, early, plot_loss_1],
                        verbose=1)

In [ ]:
from google.colab import files

files.download('/content/squeezenet_rev230131.h5')